In [96]:
import urllib.request

def gather_input_data(url, sessionId, transform=lambda x: str(x, "utf-8").strip('\n')):
    request = urllib.request.Request(url)
    request.add_header("cookie", "session={}".format(sessionId)) # Source the data directly from AoC

    values = []
    with urllib.request.urlopen(request) as data:
        for line in data:
            values.append(transform(line))

    return values

def get_data(day, year=2023):
    with open('sessionID') as f:
        sessionId = f.readlines()[0].strip()
    url = "https://adventofcode.com/%d/day/%d/input"%(year,day)
    data = gather_input_data(url, sessionId)
    return data
    

from collections import Counter, defaultdict
from itertools import groupby, pairwise
import re
from functools import lru_cache


In [114]:
@lru_cache
def recursive_arrangements(pixels, groups):
    if len(pixels) == 0:
        # e.g. "" (,)
        # e.g. "" (1,)
        return 1 if len(groups) == 0 else 0
    if pixels.startswith("."):
        # e.g. ".###?" (4,)
        # -> "###?" (4,)
        return recursive_arrangements(pixels.strip("."), groups)
    if pixels.startswith("?"):
        # e.g. "?###?" (4,)
        # 1-> ".###?" (4,)
        # 2-> "####?" (4,)
        return recursive_arrangements(pixels.replace("?", ".", 1), groups) \
            + recursive_arrangements(pixels.replace("?", "#", 1), groups)
    if pixels.startswith("#"):
        if len(groups) == 0:
            # e.g. "##" (,)
            return 0
        if len(pixels) < groups[0]:
            # e.g. "##" (3,)
            return 0
        if any(c == "." for c in pixels[0:groups[0]]):
            # e.g. "##.???" (3,1)
            return 0
        if len(groups) > 1:
            if len(pixels) < groups[0] + 1 or pixels[groups[0]] == "#":
                return 0
            # e.g. "##.???" (2,1)
            # -> "???" (1,)
            return recursive_arrangements(pixels[groups[0] + 1:], groups[1:])
        else:
            # e.g. "##.???" (2,)
            # -> ".???" (,)
            return recursive_arrangements(pixels[groups[0]:], groups[1:])

for part2 in [False, True]:
    total_arrangements = 0
    for line in get_data(12):
        flawed_map, numeric_map = line.split(" ")
        numeric_map = tuple(map(int, re.findall('\\d+', numeric_map)))
        if part2:
            flawed_map = "?".join([flawed_map] * 5)
            numeric_map = numeric_map * 5
        # left = pixels (as string), right = groups (as tuple of ints)
        total_arrangements += recursive_arrangements(flawed_map, numeric_map)
    print(total_arrangements) 


8419
160500973317706


(7, 1)